In [1]:
PARQUET = "/opt/data/target/waterbase.parquet"

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.\
        builder.\
        appName("spark2kafka_producer").\
        config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1').\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()    

In [4]:
df = spark.read.parquet(PARQUET)

In [5]:
# Work with smaller dataset
df = df.limit(100)

In [6]:
df.show(n=5)

+------------------------+------------------------------+--------------------------+-------------------------------+-------------------------+----------------------+---------+---------------------------+-----------------------+-----------------+---------------------+------------------------------------+----------------------------+------------------+-------------------------+---------------+----------------------------+------------------+---------------------------+-----------------+----------------------------+-------------------------+--------------------+-----------------------+-------+--------------------+-----------------------------+-------------------+--------------------------+--------------------+-------+
|monitoringSiteIdentifier|monitoringSiteIdentifierScheme|parameterWaterBodyCategory|observedPropertyDeterminandCode|procedureAnalysedFraction|procedureAnalysedMedia|resultUom|phenomenonTimeReferenceYear|parameterSamplingPeriod|procedureLOQValue|resultNumberOfSamples|resultQua

In [7]:
df = df.withColumn('country', df.monitoringSiteIdentifier.substr(1, 2))

In [8]:
df.show(n=5)

+------------------------+------------------------------+--------------------------+-------------------------------+-------------------------+----------------------+---------+---------------------------+-----------------------+-----------------+---------------------+------------------------------------+----------------------------+------------------+-------------------------+---------------+----------------------------+------------------+---------------------------+-----------------+----------------------------+-------------------------+--------------------+-----------------------+-------+--------------------+-----------------------------+-------------------+--------------------------+--------------------+-------+-------+
|monitoringSiteIdentifier|monitoringSiteIdentifierScheme|parameterWaterBodyCategory|observedPropertyDeterminandCode|procedureAnalysedFraction|procedureAnalysedMedia|resultUom|phenomenonTimeReferenceYear|parameterSamplingPeriod|procedureLOQValue|resultNumberOfSamples|r

In [7]:
schema = df.schema

In [8]:
print(schema)

StructType(List(StructField(monitoringSiteIdentifier,StringType,true),StructField(monitoringSiteIdentifierScheme,StringType,true),StructField(parameterWaterBodyCategory,StringType,true),StructField(observedPropertyDeterminandCode,StringType,true),StructField(procedureAnalysedFraction,StringType,true),StructField(procedureAnalysedMedia,StringType,true),StructField(resultUom,StringType,true),StructField(phenomenonTimeReferenceYear,StringType,true),StructField(parameterSamplingPeriod,StringType,true),StructField(procedureLOQValue,StringType,true),StructField(resultNumberOfSamples,StringType,true),StructField(resultQualityNumberOfSamplesBelowLOQ,StringType,true),StructField(resultQualityMinimumBelowLOQ,StringType,true),StructField(resultMinimumValue,StringType,true),StructField(resultQualityMeanBelowLOQ,StringType,true),StructField(resultMeanValue,StringType,true),StructField(resultQualityMaximumBelowLOQ,StringType,true),StructField(resultMaximumValue,StringType,true),StructField(resultQua

In [9]:
stream_df = spark.readStream.schema(schema).parquet(PARQUET)

In [10]:
stream_df.isStreaming

True

In [11]:
stream_df.selectExpr("CAST(UID AS STRING) AS key", "to_json(struct(*)) AS value").writeStream.outputMode("append").format("console").start()

In [12]:
stream_df_write = stream_df.selectExpr("CAST(UID AS STRING) AS key", "to_json(struct(*)) AS value").writeStream.outputMode("append").format("console").start()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38465)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1212, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 977,

Py4JError: PythonUtils does not exist in the JVM

In [ ]:
stream_df_write.awaitTermination()

In [13]:
query = stream_df \
  .selectExpr("CAST(UID AS STRING) AS key", "to_json(struct(*)) AS value") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("topic", "topic-demo") \
  .option("checkpointLocation", "/opt/data/checkpoint/") \
  .start() \
  .awaitTermination()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39225)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:39225)

In [14]:
read_stream_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "topic-demo") \
  .option("startingOffsets", "earliest") \
  .load()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:36539)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:36539)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:36539)